# SQLite

In [1]:
import sqlite3
import json
from typing import Dict, Any

class Database:
    def __init__(self, db_path: str):
        """
        Initialize SQLite database connection.
        :param db_path: Path to the SQLite .db file
        """
        try:
            self.connection = sqlite3.connect(db_path)
            # Enable accessing rows as dicts
            self.connection.row_factory = sqlite3.Row
            self.cursor = self.connection.cursor()
            print("🔌 Database connection successful")
        except sqlite3.Error as e:
            print(f"❌ Database connection failed: {e}")
            raise

    def _create_table_if_not_exists(self):
        """
        Ensure the user_profiles table exists with the correct schema.
        """
        try:
            self.cursor.execute(
                """
                CREATE TABLE IF NOT EXISTS user_profiles (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    age INTEGER,
                    gender TEXT,
                    marital_status TEXT,
                    education TEXT,
                    job TEXT,
                    location TEXT,
                    first_interest TEXT,
                    first_interest_percentage INTEGER,
                    second_interest TEXT,
                    second_interest_percentage INTEGER,
                    third_interest TEXT,
                    third_interest_percentage INTEGER,
                    personality_summary TEXT,
                    key_activities TEXT,
                    total_posts INTEGER,
                    top_habits TEXT,
                    top_hobby TEXT,
                    travel_indicators TEXT,
                    life_indicators TEXT,
                    spending_indicators TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
                """
            )
            self.connection.commit()
            print("✅ Table verified/created successfully")
        except sqlite3.Error as e:
            print(f"❌ Table creation failed: {e}")
            raise

    def insert_user_profile(self, profile_data: Dict[str, Any]) -> bool:
        """
        Insert a user profile into the database with transaction handling.
        :param profile_data: Dictionary containing user profile fields and interest lists
        :return: True on success, False otherwise
        """
        try:
            self._create_table_if_not_exists()

            interests = profile_data.get("top_interests", [{} for _ in range(3)])

            sql = (
                """
                INSERT INTO user_profiles (
                    first_name, last_name, age, gender, marital_status, education, job, location,
                    first_interest, first_interest_percentage,
                    second_interest, second_interest_percentage,
                    third_interest, third_interest_percentage,
                    personality_summary, key_activities, total_posts, top_habits,
                    top_hobby, travel_indicators, life_indicators, spending_indicators
                ) VALUES (
                    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
                )
                """
            )

            values = (
                profile_data.get("first_name", ""),
                profile_data.get("last_name", ""),
                profile_data.get("age"),
                profile_data.get("gender", ""),
                profile_data.get("marital_status", ""),
                profile_data.get("education", ""),
                profile_data.get("job", ""),
                profile_data.get("location", ""),
                interests[0].get("interest", ""),
                interests[0].get("percentage", 0),
                interests[1].get("interest", ""),
                interests[1].get("percentage", 0),
                interests[2].get("interest", ""),
                interests[2].get("percentage", 0),
                profile_data.get("personality_summary", ""),
                json.dumps(profile_data.get("key_activities", [])),
                profile_data.get("total_posts", 0),
                json.dumps(profile_data.get("top_habits", [])),
                profile_data.get("top_hobby", ""),
                profile_data.get("travel_indicators", ""),
                json.dumps(profile_data.get("life_indicators", [])),
                json.dumps(profile_data.get("spending_indicators", []))
            )

            self.cursor.execute(sql, values)
            self.connection.commit()
            print("✅ Profile inserted successfully")
            return True

        except sqlite3.Error as e:
            self.connection.rollback()
            print(f"❌ Database error: {e}")
            return False
        except Exception as e:
            self.connection.rollback()
            print(f"❌ Unexpected error: {e}")
            return False
        finally:
            self.close()

    def close(self):
        """
        Close the database connection and cursor.
        """
        try:
            if hasattr(self, 'cursor'):
                self.cursor.close()
            if hasattr(self, 'connection'):
                self.connection.close()
            print("🔌 Database connection closed")
        except Exception as e:
            print(f"❌ Error closing connection: {e}")


In [2]:
db = Database("test_profiles.db")


🔌 Database connection successful


In [3]:

sample_profile = {
    "first_name": "John",
    "last_name": "Doe",
    "age": 28,
    "gender": "Male",
    "marital_status": "Single",
    "education": "Bachelor's in Computer Science",
    "job": "Software Engineer",
    "location": "Cairo, Egypt",
    "top_interests": [
        {"interest": "Technology", "percentage": 70},
        {"interest": "Traveling", "percentage": 20},
        {"interest": "Reading", "percentage": 10}
    ],
    "personality_summary": "Enthusiastic and proactive learner.",
    "key_activities": ["Coding", "Blogging", "Hiking"],
    "total_posts": 45,
    "top_habits": ["Morning run", "Reading news"],
    "top_hobby": "Photography",
    "travel_indicators": "Frequent traveler",
    "life_indicators": ["Outgoing", "Family-oriented"],
    "spending_indicators": ["Budget-conscious"]
}

In [4]:
inserted = db.insert_user_profile(sample_profile)
print(f"Insert successful: {inserted}")



✅ Table verified/created successfully
✅ Profile inserted successfully
🔌 Database connection closed
Insert successful: True


In [5]:
# Verify insertion by querying directly
conn = sqlite3.connect("test_profiles.db")
conn.row_factory = sqlite3.Row
cur = conn.cursor()
cur.execute("SELECT * FROM user_profiles")
rows = cur.fetchall()
for row in rows:
    print(dict(row))
cur.close()
conn.close()

{'id': 1, 'first_name': 'John', 'last_name': 'Doe', 'age': 28, 'gender': 'Male', 'marital_status': 'Single', 'education': "Bachelor's in Computer Science", 'job': 'Software Engineer', 'location': 'Cairo, Egypt', 'first_interest': 'Technology', 'first_interest_percentage': 70, 'second_interest': 'Traveling', 'second_interest_percentage': 20, 'third_interest': 'Reading', 'third_interest_percentage': 10, 'personality_summary': 'Enthusiastic and proactive learner.', 'key_activities': '["Coding", "Blogging", "Hiking"]', 'total_posts': 45, 'top_habits': '["Morning run", "Reading news"]', 'top_hobby': 'Photography', 'travel_indicators': 'Frequent traveler', 'life_indicators': '["Outgoing", "Family-oriented"]', 'spending_indicators': '["Budget-conscious"]', 'created_at': '2025-07-09 21:52:41'}


# Mysql

In [ ]:
import pymysql
mydb = pymysql.connect(host="", user="", password="")

In [2]:
mycursor = mydb.cursor()

In [3]:
mycursor.execute("USE mydatabase")

0

In [6]:
mycursor.execute("""CREATE TABLE user_profiles (
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    age INT,
    gender VARCHAR(20),
    marital_status VARCHAR(50),
    education VARCHAR(100),
    job VARCHAR(100),
    location VARCHAR(100),
    
    -- Top interests as separate columns
    first_interest VARCHAR(50),
    first_interest_percentage INT,
    second_interest VARCHAR(50),
    second_interest_percentage INT,
    third_interest VARCHAR(50),
    third_interest_percentage INT,
    
    personality_summary TEXT,
    
    -- Activities (stored as JSON array)
    key_activities TEXT,
    
    total_posts INT,
    
    -- Habits (stored as JSON array)
    top_habits TEXT,
    
    top_hobby VARCHAR(100),
    travel_indicators VARCHAR(50),
    
    -- Life indicators (stored as JSON array)
    life_indicators TEXT,
    
    -- Spending indicators (stored as JSON array)
    spending_indicators TEXT
)""")

0

In [ ]:
from typing import Dict, Any
import pymysql
import json

class Database:
    def __init__(self, host: str, user: str, password: str, db: str):
        """Initialize the database connection"""
        self.connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            db=db,
            cursorclass=pymysql.cursors.DictCursor
        )
        self.cursor = self.connection.cursor()

    def insert_user_profile(self, profile_data: Dict[str, Any]) -> bool:
        """Insert a user profile into the database"""
        try:
            # Convert lists to JSON strings
            key_activities = json.dumps(profile_data.get("key_activities", []))
            top_habits = json.dumps(profile_data.get("top_habits", []))
            life_indicators = json.dumps(profile_data.get("life_indicators", []))
            spending_indicators = json.dumps(profile_data.get("spending_indicators", []))
            
            # Get interests from the top_interests list
            interests = profile_data.get("top_interests", [{} for _ in range(3)])
            
            sql = """
            INSERT INTO user_profiles (
                first_name, last_name, age, gender, marital_status, education, job, location,
                first_interest, first_interest_percentage,
                second_interest, second_interest_percentage,
                third_interest, third_interest_percentage,
                personality_summary, key_activities, total_posts, top_habits,
                top_hobby, travel_indicators, life_indicators, spending_indicators
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s
            )
            """

            values = (
                profile_data.get("first_name"),
                profile_data.get("last_name"),
                profile_data.get("age"),
                profile_data.get("gender"),
                profile_data.get("marital_status"),
                profile_data.get("education"),
                profile_data.get("job"),
                profile_data.get("location"),
                interests[0].get("interest"),
                interests[0].get("percentage"),
                interests[1].get("interest"),
                interests[1].get("percentage"),
                interests[2].get("interest"),
                interests[2].get("percentage"),
                profile_data.get("personality_summary"),
                key_activities,
                profile_data.get("total_posts"),
                top_habits,
                profile_data.get("top_hobby"),
                profile_data.get("travel_indicators"),
                life_indicators,
                spending_indicators
            )

            self.cursor.execute(sql, values)
            self.connection.commit()
            self.logger.info(f"Successfully inserted profile for {profile_data.get('first_name')} {profile_data.get('last_name')}")
            return True

        except pymysql.MySQLError as e:
            self.logger.error(f"Error inserting user profile: {e}")
            return False
        except Exception as e:
            self.logger.error(f"Unexpected error: {e}")
            return False
